In [12]:
import os
from dotenv import load_dotenv
load_dotenv('.flaskenv')

True

In [13]:
import pandas as pd

from service import db_functions as db
from service.ml_networks.helper import sample_hyperparameters

from probabilistic.energy_predictions import ProbabilisticEnergyPrediction
from probabilistic.probabilistic_parameters import ProbabilisticParameters

from service.helper import json, JsonEncoder, requests, DB_URL, HEADER

from helper.run_period import get_run_periods

from ml_models import get_regressor, get_scaling_parameters

In [14]:
hyperparameters_dict = {
  "num_neurons": [
    [
      20,
      50,
      100,
    ],
    [
      0,
      5,
      10,
    ]
  ],
  "reg_coef": [
    0.01,
    0.003,
    0.001,
  ],
  "learning_rate": [
    0.01,
    0.003,
    0.001,
  ]
}

In [15]:
def add_loss(hyperparameters, losses):
    hyperparameters['losses'] = losses
    hyperparameters.dropna(axis=0, inplace=True)
    hyperparameters.sort_values(by='losses', inplace=True)
    hyperparameters.index = range(len(hyperparameters))
    


def update_database(hyperparameters, network):
    data = {
        "TYPE": "UPDATE_ITEM", 
        "TABLE_NAME": "hyperparameters",
        "SET_VALUES": f"value='{json.dumps(hyperparameters, cls=JsonEncoder)}'",
        "CONDITIONS": f"name='{network}'",
    }
    response = requests.post(DB_URL, headers=HEADER, json=data).json()
    if (response["ERROR"]):
        raise ValueError(response["ERROR"])

In [ ]:
hyperparameters = sample_hyperparameters(hyperparameters_dict, 25)
losses = []

search_conditions = db.get_search_conditions('tum-gni', 'tausendpfund-gn')

project_settings = db.get_columns(search_conditions, db.PROJECT_SETTINGS,)

sampled_parameters = db.get_columns(
        search_conditions, db.SAMPLED_PARAMETERS, True)

simulation_results = db.get_columns(
    search_conditions, db.SIMULATION_RESULTS,)

parameters_df = db.get_columns(
    search_conditions, db.PARAMETERS, True)

p_parameters = ProbabilisticParameters.from_df(parameters_df)

regressor_targets = ProbabilisticEnergyPrediction.from_json(
        simulation_results).Values["Total"]

scaling_df_Y = get_scaling_parameters(regressor_targets, all_columns_equal=True)
for _, hp in hyperparameters.iterrows():
    regressor = get_regressor(hp, sampled_parameters, regressor_targets,
                                p_parameters.get_scaling_df(), scaling_df_Y)
    losses.append(regressor[1])

add_loss(hyperparameters, losses)
update_database(hyperparameters, 'regressor')

In [16]:
db.get_hyperparameters_all(network='regressor').loc[0]

num_neurons       [20, 0]
reg_coef             0.01
learning_rate        0.01
losses           0.003418
Name: 0, dtype: object

In [17]:
from ml_models import get_generator, get_scaling_layer
import numpy as np

hyperparameters = sample_hyperparameters(hyperparameters_dict, 25)
losses = []

search_conditions = db.get_search_conditions('tum-gni', 'tausendpfund-ed')

regressor_data = db.get_columns(search_conditions, db.REGRESSOR)
regressor = regressor_data[db.NETWORK]
regressor.set_weights([np.array(x) for x in regressor_data[db.WEIGHTS]])

parameters_df = db.get_columns(
    search_conditions, db.PARAMETERS, True)

p_parameters = ProbabilisticParameters.from_df(parameters_df)

scaling_data = db.get_columns(search_conditions, db.SCALING)
scaling_df_y = pd.DataFrame.from_dict(scaling_data[db.SCALING_DF_Y])

consumption_df = db.get_columns(
            search_conditions, db.CONSUMPTION, True)

_, consumption = get_run_periods(consumption_df)

for _, hp in hyperparameters.iterrows():
    targets = np.array([consumption.mean(axis=1) for _ in range(1000)])
    targets = get_scaling_layer(scaled_df=scaling_df_y)(targets).numpy()
    generator = get_generator(
                hp,
                p_parameters.get_scaling_df(),
                regressor, targets,
                error_domain=[0.25, 0.75])
    losses.append(generator[1])

add_loss(hyperparameters, losses)

update_database(hyperparameters, 'generative')

Epoch 1/1000
1/1 [==============================] - 1s 1s/step - loss: 1.3288 - val_loss: 1.2674
Epoch 2/1000
1/1 [==============================] - 0s 58ms/step - loss: 1.2689 - val_loss: 1.2099
Epoch 3/1000
1/1 [==============================] - 0s 73ms/step - loss: 1.2112 - val_loss: 1.1544
Epoch 4/1000
1/1 [==============================] - 0s 71ms/step - loss: 1.1556 - val_loss: 1.1012
Epoch 5/1000
1/1 [==============================] - 0s 64ms/step - loss: 1.1022 - val_loss: 1.0500
Epoch 6/1000
1/1 [==============================] - 0s 60ms/step - loss: 1.0509 - val_loss: 1.0008
Epoch 7/1000
1/1 [==============================] - 0s 71ms/step - loss: 1.0016 - val_loss: 0.9536
Epoch 8/1000
1/1 [==============================] - 0s 64ms/step - loss: 0.9544 - val_loss: 0.9084
Epoch 9/1000
1/1 [==============================] - 0s 65ms/step - loss: 0.9091 - val_loss: 0.8651
Epoch 10/1000
1/1 [==============================] - 0s 58ms/step - loss: 0.8657 - val_loss: 0.8236
Epoch 11/10

In [18]:
hyperparameters

,num_neurons,reg_coef,learning_rate,losses
0,"(50, 0)",0.010,0.003,0.076562
1,"(100, 0)",0.003,0.003,0.076705
2,"(50, 10)",0.010,0.010,0.077472
3,"(100, 10)",0.010,0.010,0.077816
4,"(100, 0)",0.010,0.010,0.079013
5,"(100, 0)",0.003,0.010,0.079403
6,"(100, 10)",0.010,0.003,0.079801
7,"(100, 10)",0.001,0.003,0.080513
8,"(50, 5)",0.010,0.003,0.081580
9,"(50, 0)",0.003,0.001,0.089272


In [19]:
hyperparameters = sample_hyperparameters(hyperparameters_dict, 25)
losses = []

search_conditions = db.get_search_conditions('tum-gni', 'tausendpfund-in')

project_settings = db.get_columns(search_conditions, db.PROJECT_SETTINGS,)

sampled_parameters = db.get_columns(
        search_conditions, db.SAMPLED_PARAMETERS, True)

simulation_results = db.get_columns(
    search_conditions, db.SIMULATION_RESULTS,)

parameters_df = db.get_columns(
    search_conditions, db.PARAMETERS, True)

p_parameters = ProbabilisticParameters.from_df(parameters_df)

regressor_targets = ProbabilisticEnergyPrediction.from_json(
        simulation_results).Values["Total"]

scaling_df_X = get_scaling_parameters(regressor_targets, all_columns_equal=True)
for _, hp in hyperparameters.iterrows():
    regressor = get_regressor(hp, regressor_targets, sampled_parameters, 
                                scaling_df_X, p_parameters.get_scaling_df(), inverted=True)
    losses.append(regressor[1])

add_loss(hyperparameters, losses)
update_database(hyperparameters, 'inverted')

Epoch 1/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1896 - val_loss: 0.1821
Epoch 2/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.1885 - val_loss: 0.1812
Epoch 3/1000
1/1 [==============================] - 0s 64ms/step - loss: 0.1874 - val_loss: 0.1802
Epoch 4/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.1863 - val_loss: 0.1792
Epoch 5/1000
1/1 [==============================] - 0s 53ms/step - loss: 0.1852 - val_loss: 0.1783
Epoch 6/1000
1/1 [==============================] - 0s 50ms/step - loss: 0.1841 - val_loss: 0.1773
Epoch 7/1000
1/1 [==============================] - 0s 66ms/step - loss: 0.1831 - val_loss: 0.1764
Epoch 8/1000
1/1 [==============================] - 0s 52ms/step - loss: 0.1820 - val_loss: 0.1755
Epoch 9/1000
1/1 [==============================] - 0s 52ms/step - loss: 0.1809 - val_loss: 0.1746
Epoch 10/1000
1/1 [==============================] - 0s 51ms/step - loss: 0.1799 - val_loss: 0.1736
Epoch 11/10

In [22]:
db.get_hyperparameters_all('generative')

,num_neurons,reg_coef,learning_rate,losses
0,"[50, 0]",0.010,0.003,0.076562
1,"[100, 0]",0.003,0.003,0.076705
2,"[50, 10]",0.010,0.010,0.077472
3,"[100, 10]",0.010,0.010,0.077816
4,"[100, 0]",0.010,0.010,0.079013
5,"[100, 0]",0.003,0.010,0.079403
6,"[100, 10]",0.010,0.003,0.079801
7,"[100, 10]",0.001,0.003,0.080513
8,"[50, 5]",0.010,0.003,0.081580
9,"[50, 0]",0.003,0.001,0.089272
